In [28]:
################### MODELO USADO ORIGINALMENTE 
from tqdm import tqdm
from Bio import PDB
import os
import pandas as pd
from pathlib import Path


DATA_DIR_PATH = Path('/home/douglas/carboanalysis/data/unzipped/')
CARBO_LIST_PATH = Path('/home/douglas/carboanalysis/carboanalysis/pdb/dicts/CCD_carbohydrate_list.tsv')

def get_all_covalent_contacts(mmcif_file):
    parser = PDB.MMCIFParser(QUIET=True)
    structure = parser.get_structure("structure", DATA_DIR_PATH / mmcif_file)

    carbo_dict = pd.read_csv(CARBO_LIST_PATH, sep="\t", header=None, names=['carbo_id', 'release_status'])
    carbohydrate_residues = carbo_dict["carbo_id"].values

    protein_atoms = []
    carbohydrate_atoms = []
    carbohydrate_chains = set()

    for model in structure:
        for chain in model:
            for residue in chain:
                resname = residue.get_resname().strip()
                if resname in carbohydrate_residues:
                    carbohydrate_chains.add(chain)
                    for atom in residue.get_atoms():
                        carbohydrate_atoms.append((residue, atom))
                elif PDB.is_aa(residue):
                    for atom in residue.get_atoms():
                        protein_atoms.append((residue, atom))

    results = set()
    covalent_chains = set()
    n_glyco_chains = set()
    o_glyco_chains = set()
    for protein_residue, protein_atom in protein_atoms:
        for carb_residue, carb_atom in carbohydrate_atoms:
            dist = protein_atom - carb_atom
            # (entry, carb_resname, carb_residue_number, carb_chain_id)

            results.add((mmcif_file.replace('.cif', ''),  # sem extensão
                            carb_residue.get_resname(),
                            carb_residue.get_id()[1],
                            carb_residue.get_parent().get_id()))
            if dist <= 2.0:
                covalent_chains.add(carb_residue.get_parent().get_id())
                print(carb_residue.get_parent().get_id())

                carb_atom_name = carb_atom.get_name()
                prot_atom_name = protein_atom.get_name()
                prot_resname = protein_atom.get_parent().get_resname()

                if prot_resname == 'ASN' and prot_atom_name.startswith('N'):
                    n_glyco_chains.add(carb_residue.get_parent().get_id())
                elif (prot_resname == 'SER' or prot_resname == 'THR') and prot_atom_name.startswith('O'):
                    o_glyco_chains.add(carb_residue.get_parent().get_id())

                        


    if results:
        carbohydrate_residues_df = pd.DataFrame(results, columns=['entry', 'carb_resname', 'carb_residue_number', 'carb_chain_id'])
        carbohydrate_residues_df['covalent'] = False
        carbohydrate_residues_df.loc[carbohydrate_residues_df['carb_chain_id'].isin(covalent_chains), 'covalent'] = True
        carbohydrate_residues_df['glycosylation'] = None
        carbohydrate_residues_df.loc[carbohydrate_residues_df['carb_chain_id'].isin(n_glyco_chains), 'glycosylation'] = 'N-Glycosylation'
        carbohydrate_residues_df.loc[carbohydrate_residues_df['carb_chain_id'].isin(o_glyco_chains), 'glycosylation'] = 'O-Glycosylation'
        
        return carbohydrate_residues_df
    else:
        return pd.DataFrame(columns=['entry', 'carb_resname', 'carb_residue_number', 'carb_chain_id', 'covalent'])



entry_list = ['2wmg.cif', '4bfn.cif', '1arp.cif']

all_results = []
for entry in tqdm(entry_list, desc='Caracterizando Interações'):
    df = get_all_covalent_contacts(entry)
    all_results.append(df)

final_df = pd.concat(all_results, ignore_index=True)
final_df.to_csv('carbohydrate_interactions_debug_2.csv', index=False)

Caracterizando Interações:   0%|          | 0/3 [00:00<?, ?it/s]

Caracterizando Interações:  67%|██████▋   | 2/3 [00:01<00:00,  1.23it/s]

B


Caracterizando Interações: 100%|██████████| 3/3 [00:02<00:00,  1.23it/s]


In [21]:
from Bio import PDB
import os
import pandas as pd
from pathlib import Path

DATA_DIR_PATH = Path('/home/douglas/carboanalysis/data/unzipped/')
CARBO_LIST_PATH = Path('/home/douglas/carboanalysis/carboanalysis/pdb/dicts/CCD_carbohydrate_list.tsv')

def get_all_covalent_contacts(mmcif_file):
    parser = PDB.MMCIFParser(QUIET=True)
    structure = parser.get_structure("structure", DATA_DIR_PATH / mmcif_file)

    carbo_dict = pd.read_csv(CARBO_LIST_PATH, sep="\t", header=None, names=['carbo_id', 'release_status'])
    carbohydrate_residues = carbo_dict["carbo_id"].values

    protein_atoms = []
    carbohydrate_atoms = []
    carbohydrate_chains = set()

    for model in structure:
        for chain in model:
            for residue in chain:
                resname = residue.get_resname().strip()
                if resname in carbohydrate_residues:
                    carbohydrate_chains.add(chain)
                    for atom in residue.get_atoms():
                        carbohydrate_atoms.append((residue, atom))
                elif PDB.is_aa(residue):
                    for atom in residue.get_atoms():
                        protein_atoms.append((residue, atom))

    results = set()
    covalent_chains = set()
    for protein_residue, protein_atom in protein_atoms:
        for carb_residue, carb_atom in carbohydrate_atoms:
            dist = protein_atom - carb_atom
            # (entry, carb_resname, carb_residue_number, carb_chain_id) 'entry', 'carb_resname', 'carb_residue_number', 'carb_chain_id', 'carb_atom', 'prot_atom', 'prot'

            # results = {'entry': mmcif_file.replace('.cif', ''),  # sem extensão
            #                 'carb_resname': carb_residue.get_resname(),
            #                 'carb_residue_number': carb_residue.get_id()[1],
            #                 'carb_chain_id': carb_residue.get_parent().get_id(),
            #                 'carb_atom': carb_atom,
            #                 'prot_atom': protein_atom,
            #                 'prot': protein_atom.get_parent()}

            data = (
                mmcif_file.replace('.cif', ''),  # sem extensão
                carb_residue.get_resname(),
                carb_residue.get_id()[1],
                carb_residue.get_parent().get_id(),
                carb_atom,
                protein_atom,
                protein_atom.get_parent()
            )
            
            data = (mmcif_file.replace('.cif', ''),  # sem extensão
                            carb_residue.get_resname(),
                            carb_residue.get_id()[1],
                            carb_residue.get_parent().get_id())

            # results.add((mmcif_file.replace('.cif', ''),  # sem extensão
            #                 carb_residue.get_resname(),
            #                 carb_residue.get_id()[1],
            #                 carb_residue.get_parent().get_id(),
            #                 carb_atom,
            #                 protein_atom,
            #                 protein_atom.get_parent()
            #                 ))
            
            if dist <= 2.0:
                covalent_chains.add(carb_residue.get_parent().get_id())
                #data = data + (True,)
            else:
                #data = data + (False,) 
                pass
            
            results.add(data)

    if results:
        carbohydrate_residues_df = pd.DataFrame(results, columns=['entry', 'carb_resname', 'carb_residue_number', 'carb_chain_id'])#, 'carb_atom', 'prot_atom', 'prot', 'is_residue_covalent'])
        carbohydrate_residues_df['is_chain_covalent'] = False
        carbohydrate_residues_df.loc[carbohydrate_residues_df['carb_chain_id'].isin(covalent_chains), 'is_chain_covalent'] = True
        
        return carbohydrate_residues_df
    else:
        return pd.DataFrame(columns=['entry', 'carb_resname', 'carb_residue_number', 'carb_chain_id', 'carb_atom', 'prot_atom', 'prot', 'is_residue_covalent', 'is_chain_covalent'])


entry_list =  ['2wmg.cif', '4bfn.cif', '1arp.cif']

all_results = []
for entry in entry_list:
    df = get_all_covalent_contacts(entry)
    all_results.append(df)

final_df = pd.concat(all_results, ignore_index=True)
final_df.to_csv('carbohydrate_interactions_debug.csv', index=False)

In [19]:
import pandas as pd

carbo_df = pd.read_csv('/home/douglas/carboanalysis/carboanalysis/pdb/dataframes/carbo_entrys_res_owab_filtered_nd.txt', names=['entry'])
carbos = carbo_df['entry'].values
print(carbos)

['4bm7.cif' '4pik.cif' '3wli.cif' ... '2rgm.cif' '4k60.cif' '2obs.cif']


In [1]:
import pandas as pd

#Sugars
interactions_df = pd.read_csv('carbohydrate_interactions.csv')
interactions = interactions_df['covalent'].value_counts()
sugar_df = pd.DataFrame(interactions)
sugar_df['percent'] = (sugar_df['count']/sugar_df['count'].sum() * 100).round(2)

#Chains
unique_chains = interactions_df.groupby(['entry', 'carb_chain_id'])['covalent'].any().reset_index()
total_chains = len(unique_chains)
covalent_chains = unique_chains['covalent'].sum()
non_covalent_chains = total_chains - covalent_chains
chain_df = pd.DataFrame({'chain_interaction': ['covalent', 'non_covalent'], 'count': [covalent_chains, non_covalent_chains]})
chain_df['percent'] = (chain_df['count']/chain_df['count'].sum() * 100).round(2)